In [76]:
from toolz import itertoolz, pipe
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [77]:
%memit np.zeros((1000, 1000))

peak memory: 343.57 MiB, increment: 0.04 MiB


In [78]:
from random_data import sequence_generator
x = list(sequence_generator(1000000, 5, 15 ))

KeyboardInterrupt: 

In [ ]:
from df import get_edge_dataframe
df = get_edge_dataframe(x)



In [ ]:
# head of the dataframe
df.head()

In [ ]:
%memit get_edge_dataframe(x)

In [ ]:
%time get_edge_dataframe(x)

In [85]:
import dask.dataframe as dd
import pandas as pd
import random
import string
from collections import defaultdict
from dask import bag as db
from itertools import combinations
from dask.distributed import Client

def subsequence_generator(sequence):
    for ss in combinations(sequence, len(sequence) - 1):
        yield ''.join(ss)

def get_subsequence_dict(sequence):
    subsequence_dict = defaultdict(list)
    for subseq in subsequence_generator(sequence):
        subsequence_dict[subseq].append(sequence)
    return subsequence_dict

def pair_amino_acid_keys(subsequence_dict):
    pairs = set()
    for subseq, sequences in subsequence_dict.items():
        unique_sequences = list(set(sequences))  # Convert to list to maintain order
        if len(unique_sequences) > 1:  # If more than one unique amino_acid_key share this subsequence
            # Generate all possible pairs, sort them to treat (A, B) and (B, A) as the same pair, then add to set
            for pair in combinations(unique_sequences, 2):
                if pair[0] != pair[1]:  # Exclude pairs with same sequences
                    pairs.add(tuple(sorted(pair)))
    return list(pairs)

def get_pairs(amino_acid_keys):
    bag = db.from_sequence(amino_acid_keys, npartitions=10)
    subsequence_dict = bag.map(get_subsequence_dict).compute()
    merged_dict = defaultdict(list)
    for d in subsequence_dict:
        for key, value in d.items():
            merged_dict[key].extend(value)
    pairs = pair_amino_acid_keys(merged_dict)
    return pairs

def get_edge_dataframe(sequences):
    edges = get_pairs(sequences)
    edge_df = pd.DataFrame(edges, columns=['seq1', 'seq2'])
    return edge_df

# Dask client for parallel processing
client = Client()

# Generate amino acid sequences

# Get the edge dataframe
edge_df = get_edge_dataframe(amino_acid_keys)

# Convert the pandas DataFrame to a Dask DataFrame
dask_df = dd.from_pandas(edge_df, npartitions=10)

# Display the DataFrame
print(dask_df.head())

# Close the client
client.close()


C:\Users\User\anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53412 instead
  warnings.warn(


         seq1        seq2
0  BCBCABADCA  CDBCABADCA
1  BDBDDCDCAA  DBDDDCDCAA
2  DCCCACCADA  DCCCBACADA
3  ACCCCABABB  CCCBCABABB
4  BABADCADAD  BBADCBADAD


In [86]:
import random
import string
from collections import defaultdict
from dask import bag as db
from itertools import combinations
from dask.distributed import Client

def subsequence_generator(sequence):
    for ss in combinations(sequence, len(sequence) - 1):
        yield ''.join(ss)

def get_subsequence_dict(sequence):
    subsequence_dict = defaultdict(list)
    for subseq in subsequence_generator(sequence):
        subsequence_dict[subseq].append(sequence)
    return subsequence_dict

def pair_amino_acid_keys(subsequence_dict):
    pairs = set()
    for subseq, sequences in subsequence_dict.items():
        unique_sequences = list(set(sequences))  # Convert to list to maintain order
        if len(unique_sequences) > 1:  # If more than one unique amino_acid_key share this subsequence
            # Generate all possible pairs, sort them to treat (A, B) and (B, A) as the same pair, then add to set
            for pair in combinations(unique_sequences, 2):
                if pair[0] != pair[1]:  # Exclude pairs with same sequences
                    pairs.add(tuple(sorted(pair)))
    return list(pairs)

def get_pairs(amino_acid_keys):
    bag = db.from_sequence(amino_acid_keys, npartitions=10)
    subsequence_dict = bag.map(get_subsequence_dict).compute()
    merged_dict = defaultdict(list)
    for d in subsequence_dict:
        for key, value in d.items():
            merged_dict[key].extend(value)
    pairs = pair_amino_acid_keys(merged_dict)
    return pairs

# Dask client for parallel processing
client = Client()

amino_acid_keys = [''.join(random.choices(string.ascii_uppercase[:4], k=10)) for _ in range(10000)]

# Get the edge pairs
edge_pairs = get_pairs(amino_acid_keys)

for pair in edge_pairs:
    print(pair)

# Close the client
client.close()


C:\Users\User\anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53621 instead
  warnings.warn(


('CDCACABAAD', 'CDCACBADAD')
('DBABDDCBAA', 'DBBDDCBACA')
('BBDAACBDDC', 'BDACACBDDC')
('ADDCBCCCBD', 'ADDCBDCCBD')
('DBCADADDBA', 'DBCADDBDBA')
('BADDCCDBBD', 'BDDBCCDBBD')
('BDDDBDCBBB', 'BDDDDCBBCB')
('BDDBBCBACD', 'BDDBCBBACD')
('DBDADCAAAD', 'DDADCADAAD')
('BCDABBBDAC', 'BDCDBBBDAC')
('ABCADCDADA', 'ABCCADCADA')
('DDCDDCDBBA', 'DDDDACDBBA')
('AACDCDDABD', 'AACDDDABAD')
('DCDCAABAAA', 'DCDCAACAAA')
('AACCADACBD', 'AACCCADACB')
('DCBCBABDAC', 'DCCBADBDAC')
('CDDBDDBDAD', 'CDDBDDDCAD')
('DBBCCDADBB', 'DBDBCDADBB')
('DBBBBCBDAB', 'DBBBCBDACB')
('DACCABBABB', 'DACCCBBABB')
('CDCBAACDCC', 'CDDCAACDCC')
('DBABCBACDC', 'DBCABCBCDC')
('DACBADBDCA', 'DDACBADBDC')
('DABCBBBDBA', 'DACBBBBDBA')
('ACBDBCDADD', 'BACBDBCADD')
('ABDCBCBDBA', 'DABDCBCBBA')
('BACABBACDC', 'BACABBADCD')
('ACBABADDBA', 'ACBABDBDBA')
('BADCDBDAAA', 'BADDBDAAAD')
('CDAACCDDCC', 'CDACACCDDC')
('BDCBDBACDA', 'BDDCBDBACA')
('DCCDABDDCD', 'DCDCDABDCD')
('ACBACDDCBD', 'ACBACDDCDC')
('BBAAAAADCB', 'DBBAAAAACB')
('BBDADBABAC',

In [ ]:
from dask.distributed import Client

client = Client()  # start a local Dask client


In [ ]:
# ... your code
# ... your code
subsequence_dict = get_subsequence_dict(amino_acid_keys)  # Generate subsequence dict
pairs = client.compute(pair_amino_acid_keys(subsequence_dict))  # Use Dask's compute
pairs = pairs.result()  # To get the result when it's ready

pairs = client.compute(pair_amino_acid_keys(merged_dict))  # Use Dask's compute
pairs = pairs.result()  # To get the result when it's ready
